# Day 21
## Part 1
I thought this was a reasonably simple dynamic programming problem but I got in a real mess. Wish I hadn't started.

In [74]:
from functools import cache
from dataclasses import dataclass

@dataclass(eq=True, frozen=True)
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

DIRECTIONS = {N, E, S, W}

def manhattan(p1, p2):
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)


def keypad(rows):
    pad = {}
    for y, line in enumerate(reversed(rows)):
        for x, c in enumerate(line):
            pad[c] = Point(x, y)
    return pad

@cache
def nkeypad():
    return keypad(["789", "456", "123", " 0A"])

@cache
def dkeypad():
    return keypad([" ^A", "<v>"])

def dir_presses(p_from, p_to, pad):
    p_from = pad[p_from]
    p_to = pad[p_to]
    d = p_to - p_from
    dx = 1 if d.x > 0 else -1
    dy = 1 if d.y > 0 else -1
    xs = list(range(p_from.x, p_to.x + dx, dx))
    ys = list(range(p_from.y, p_to.y + dy, dy))
    xstr = ("<" if dx == -1 else ">") * abs(d.x)
    ystr = ("v" if dy == -1 else "^") * abs(d.y)

    result = set()
    if not (pad[" "].y == p_to.y and pad[" "].x in xs):
        result.add(ystr + xstr + "A")
    if not (pad[" "].x == p_to.x and pad[" "].y in ys):
        result.add(xstr + ystr + "A")
    return result

@cache
def d_dir_presses(p_from, p_to):
    return dir_presses(p_from, p_to, dkeypad())

def min_path(code):
    result = 0
    for p1, p2 in zip("A" + code, code):
        result += min(
            min_path_d(s, 2) 
            for s in dir_presses(p1, p2, nkeypad())
        )
    return result

@cache
def min_path_d(ds, robot_no):
    if robot_no == 0:
        return len(ds)
    result = 0
    for p1, p2 in zip("A" + ds, ds):
        result += min(
            min_path_d(s, robot_no - 1) 
            for s in d_dir_presses(p1, p2)
        )
    return result

min_path("029A")

68

In [75]:
def parse_data(s):
    return s.strip().splitlines()

test_data = parse_data("""029A
980A
179A
456A
379A""")

def part_1(data):
    return sum(
        int(code[:-1]) * min_path(code) 
        for code in data
    )

for code in test_data:
    print(code, min_path(code))

029A 68
980A 60
179A 68
456A 64
379A 64


In [77]:
part_1(test_data)

126384

In [78]:
data = parse_data(open("input").read())

part_1(data)

215374

## Part 2

In [80]:
def min_path_2(code):
    result = 0
    for p1, p2 in zip("A" + code, code):
        result += min(
            min_path_d(s, 25) 
            for s in dir_presses(p1, p2, nkeypad())
        )
    return result

def part_2(data):
    return sum(
        int(code[:-1]) * min_path_2(code) 
        for code in data
    )

part_2(data)

260586897262600